In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Embedding module

I will show you the examples using OpenAI embedding API. Gemini embedding API is also available in the same manner. See [embedding_text.ipynb](./06.embedding_text.ipynb).

In [3]:
from langrila.openai import OpenAIEmbeddingModule

In [4]:
embedder_openai = OpenAIEmbeddingModule(
    api_key_env_name="API_KEY",
    model_name="text-embedding-3-small",
    dimensions=1536,
)

Here is sample documents and metadatas.

In [5]:
documents = [
    "Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.",
    "LangChain is a framework for developing applications powered by language models.",
    "LlamaIndex (GPT Index) is a data framework for your LLM application.",
]

# [Optional] metadatas
metadatas = [
    {"github_url": "https://github.com/taikinman/langrila"},
    {"github_url": "https://github.com/langchain-ai/langchain"},
    {"github_url": "https://github.com/run-llama/llama_index"},
]

# For Qdrant

Create collection at the first step.

In [ ]:
from qdrant_client import models
from langrila.database.qdrant import QdrantLocalCollectionModule

collection = QdrantLocalCollectionModule(
    persistence_directory="./qdrant_test",
    collection_name="sample_collection",
    embedder=embedder_openai,
    vectors_config=models.VectorParams(
        size=1536,
        distance=models.Distance.COSINE,
    ),
)

# Create collection
collection.run(
    documents=documents, metadatas=metadatas
)

Create collection sample_collection.
100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


Instantiate retrieval module.

In [ ]:
retriever = collection.as_retriever(
    n_results=2,
    score_threshold=0.3,  # The meaning of the score depends on the distance function
)

You can initiate retrieval module directly like this.

In [13]:
from langrila.database.qdrant import QdrantLocalRetrievalModule

retriever = QdrantLocalRetrievalModule(
    persistence_directory="./qdrant_test",
    collection_name="sample_collection",
    embedder=embedder_openai,
    n_results=2,
    score_threshold=0.3,
)

Query index.

In [14]:
query = "What is Langrila?"

retrieval_results = retriever.run(query=query, filter=None)

Retrieve from collection sample_collection...


Return is `RetrievalResults` instance which is a pydantic model.

In [15]:
retrieval_results

RetrievalResults(ids=[0, 1], documents=['Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.', 'LangChain is a framework for developing applications powered by language models.'], metadatas=[{'github_url': 'https://github.com/taikinman/langrila', 'document': 'Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.'}, {'github_url': 'https://github.com/langchain-ai/langchain', 'document': 'LangChain is a framework for developing applications powered by language models.'}], scores=[0.5302091343925472, 0.39182931721751785], collections=['sample_collection', 'sample_collection'], usage=Usage(prompt_tokens=6, completion_tokens=0, total_tokens=6))

In [16]:
retrieval_results.model_dump()

{'ids': [0, 1],
 'documents': ['Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.',
  'LangChain is a framework for developing applications powered by language models.'],
 'metadatas': [{'github_url': 'https://github.com/taikinman/langrila',
   'document': 'Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.'},
  {'github_url': 'https://github.com/langchain-ai/langchain',
   'document': 'LangChain is a framework for developing applications powered by language models.'}],
 'scores': [0.5302091343925472, 0.39182931721751785],
 'collections': ['sample_collection', 'sample_collection'],
 'usage': {'model_name': 'text-embedding-3-small',
  'prompt_tokens': 6,
  'completion_tokens': 0}}

Score in the result is similarity for Qdrant.

Qdrant server is also supported by QdrantRemoteCollectionModule and QdrantRemoteRetrievalModule. Here is a basic example using docker which app container and qdrant container are bridged by same network.

In [ ]:
from qdrant_client import models

from langrila.database.qdrant import QdrantRemoteCollectionModule

#######################
# create collection
#######################

collection = QdrantRemoteCollectionModule(
    url="http://qdrant",
    port="6333",
    collection_name="sample",
    embedder=embedder_openai,
    vectors_config=models.VectorParams(
        size=1536,
        distance=models.Distance.COSINE,
    ),
)


await collection.arun(
    documents=documents,
    metadatas=metadatas,
)

Create collection sample.
100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


Instantiate retriever.

In [8]:
retriever = collection.as_retriever(
    n_results=2,
    score_threshold=0.3,  # The meaning of the score depends on the distance function
)


# You can also use the retrieval module directly
# from langrila.database.qdrant import QdrantRemoteRetrievalModule

# retriever = QdrantRemoteRetrievalModule(
#     url="http://qdrant",
#     port="6333",
#     collection_name="sample",
#     embedder=embedder_openai,
#     n_results=2,
#     score_threshold=0.3,
# )

Query.

In [9]:
query = "What is Langrila?"

retrieval_results = await retriever.arun(query=query, filter=None)

retrieval_results

Retrieve from collection sample...


RetrievalResults(ids=[0, 1], documents=['Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.', 'LangChain is a framework for developing applications powered by language models.'], metadatas=[{'github_url': 'https://github.com/taikinman/langrila', 'document': 'Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.'}, {'github_url': 'https://github.com/langchain-ai/langchain', 'document': 'LangChain is a framework for developing applications powered by language models.'}], scores=[0.5301694, 0.3918205], collections=['sample', 'sample'], usage=Usage(prompt_tokens=6, completion_tokens=0, total_tokens=6))

In [10]:
retrieval_results.model_dump()

{'ids': [0, 1],
 'documents': ['Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.',
  'LangChain is a framework for developing applications powered by language models.'],
 'metadatas': [{'github_url': 'https://github.com/taikinman/langrila',
   'document': 'Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.'},
  {'github_url': 'https://github.com/langchain-ai/langchain',
   'document': 'LangChain is a framework for developing applications powered by language models.'}],
 'scores': [0.5301694, 0.3918205],
 'collections': ['sample', 'sample'],
 'usage': {'model_name': 'text-embedding-3-small',
  'prompt_tokens': 6,
  'completion_tokens': 0}}

# For Chroma

CollectionModule and RetrievalModule for chroma has the almost same interface.

In [ ]:
from langrila.database.chroma import ChromaLocalCollectionModule, ChromaLocalRetrievalModule

#######################
# create collection
#######################

collection = ChromaLocalCollectionModule(
    persistence_directory="./chroma_test",
    collection_name="sample",
    embedder=embedder_openai,
)

collection.run(documents=documents, metadatas=metadatas) # metadatas could also be used

# #######################
# # retrieval
# #######################

# In the case collection was already instantiated
retriever = collection.as_retriever(n_results=2, score_threshold=0.7)

# # In the case collection was not instantiated
# retriever = ChromaLocalRetrievalModule(
#     embedder=embedder_openai,
#     persistence_directory="./chroma_test",
#     collection_name="sample",
#     n_results=2,
#     score_threshold=0.7,
# )

query = "What is Langrila?"
retrieval_result = retriever.run(query, filter=None)

# show result
retrieval_result.model_dump()

Create collection sample.
100%|██████████| 1/1 [00:00<00:00,  1.24it/s]
Retrieve from collection sample...


{'ids': [0, 1],
 'documents': ['Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.',
  'LangChain is a framework for developing applications powered by language models.'],
 'metadatas': [{'document': 'Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.',
   'github_url': 'https://github.com/taikinman/langrila'},
  {'document': 'LangChain is a framework for developing applications powered by language models.',
   'github_url': 'https://github.com/langchain-ai/langchain'}],
 'scores': [0.4698306096647241, 0.6081795317291985],
 'collections': ['sample', 'sample'],
 'usage': {'model_name': 'text-embedding-3-small',
  'prompt_tokens': 6,
  'completion_tokens': 0}}

Please be careful for the score because chromadb retrieve index records based on distance in default, so the `scores` in `RetrievalResults` is the distance instead of similarity.

HttpClient is also supported by `ChromaRemoteCollectionModule` and `ChromaRemoteRetrievalModule`. Here is a basic example using local server running with `chroma run` command.

In [ ]:
from langrila.database.chroma import ChromaRemoteCollectionModule

#######################
# create collection
#######################

collection = ChromaRemoteCollectionModule(
    host="localhost",
    port="8000",
    collection_name="chroma_sample",
    embedder=embedder_openai,
)

await collection.arun(documents=documents, metadatas=metadatas)

Create collection chroma_sample.
100%|██████████| 1/1 [00:00<00:00,  1.77it/s]


Instantiate retriever.

In [9]:
retriever = collection.as_retriever(n_results=2, score_threshold=0.7)

Query.

In [10]:
query = "What is Langrila?"

retrieval_result = await retriever.arun(query, filter=None)
retrieval_result.model_dump()

Retrieve from collection chroma_sample...


{'ids': [0, 1],
 'documents': ['Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.',
  'LangChain is a framework for developing applications powered by language models.'],
 'metadatas': [{'document': 'Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.',
   'github_url': 'https://github.com/taikinman/langrila'},
  {'document': 'LangChain is a framework for developing applications powered by language models.',
   'github_url': 'https://github.com/langchain-ai/langchain'}],
 'scores': [0.4697908339701269, 0.6081706594022143],
 'collections': ['chroma_sample', 'chroma_sample'],
 'usage': {'model_name': 'text-embedding-3-small',
  'prompt_tokens': 6,
  'completion_tokens': 0}}

# For usearch

Usearch is a smaller & faster single-file similarity search & clustering engine for vectors. This vector db doesn't support metadata storing, so SQLite db is automatically created to store metadatas.

In [6]:
from langrila.database.usearch import UsearchLocalCollectionModule, UsearchLocalRetrievalModule

#######################
# create collection
#######################

collection = UsearchLocalCollectionModule(
    persistence_directory="./usearch_test",
    collection_name="sample",
    embedder=embedder_openai,
    dtype = "f16",
    ndim = 1536,
    connectivity = 16,
    expansion_add = 128,
    expansion_search = 64,
)

# Strongly recommended because search result may be different when new vectors are inserted after existing vectors are removed.
# Instead, rebuilding the index is recommended using `delete_collection` before upserting.
# Or use exact search to avoid this issue when search time.
collection.delete_collection()

collection.run(documents=documents) # metadatas could also be used. 

# #######################
# # retrieval
# #######################

# In the case collection was already instantiated
retriever = collection.as_retriever(n_results=2, score_threshold=0.7)

# # In the case collection was not instantiated
# retriever = UsearchLocalRetrievalModule(
#     embedder=embedder_openai,
#     persistence_directory="./usearch_test",
#     collection_name="sample",
#     dtype = "f16",
#     ndim=1536,
#     connectivity = 16,
#     expansion_add = 128,
#     expansion_search = 64,
#     n_results=2,
#     score_threshold=0.5,
# )

100%|██████████| 1/1 [00:00<00:00,  1.52it/s]


Query.

In [7]:
query = "What is Langrila?"
retrieval_result = retriever.run(query, filter=None, exact=False)

# show result
retrieval_result.model_dump()

Retrieve from collection sample...


{'ids': [0, 1],
 'documents': ['Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.',
  'LangChain is a framework for developing applications powered by language models.'],
 'metadatas': [{'document': 'Langrila is a useful tool to use ChatGPT with OpenAI API or Azure in an easy way.'},
  {'document': 'LangChain is a framework for developing applications powered by language models.'}],
 'scores': [0.4698024392127991, 0.608174204826355],
 'collections': ['sample', 'sample'],
 'usage': {'model_name': 'text-embedding-3-small',
  'prompt_tokens': 6,
  'completion_tokens': 0}}

For usearch, score is distance, not similarity.